# Pushing metadata to an FDP

This notebook can be used to manually push DCAT concepts according to the Health-RI Core v2 to a FAIR Data Point. 
If you are looking for interactive explanations, please see the 'Documentation' notebooks.

**Prerequisites:** To execute this notebook in full one needs to have a running FAIR Data Point (FDP) instance with an active write access account.
This notebook is written for the reference implementation, FAIR Data Point version 1.16 with the [Health-RI Core v2 SHACL shapes](https://github.com/Health-RI/health-ri-metadata/tree/develop/Formalisation(shacl)/Core/FairDataPointShape).

## Imports and setup

In [1]:
from typing import List, Union
from pprint import pprint

from rdflib import URIRef, DCTERMS
from pydantic import AnyHttpUrl, Field, field_validator

from getpass import getpass
import dateutil.parser as parser

from fairclient.fdpclient import FDPClient

from sempyro import LiteralField
from sempyro.hri_dcat import (
    HRICatalog, 
    HRIDataset, 
    HRIVCard, 
    HRIAgent, 
    HRIDistribution,
    HRIDataService,
    HRIDatasetSeries,
    HRIProject,
    HRIStudy
)
from sempyro.utils.validator_functions import force_literal_field

/Users/alexanderharms/.pyenv/versions/sempyro310/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: [<class 'pydantic.networks.AnyHttpUrl'>] is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/Users/alexanderharms/Documents/Projects/SeMPyRO/SeMPyRO/sempyro/hri_dcat/hri_project.py:40: UserWarning: Code: resource is not defined in namespace DCAT
  "rdf_term": DCAT.resource,


In [2]:
fdp_base=input("Enter base link to FDP: ").rstrip("/")
username=input("Enter username: ")
password = getpass(prompt="Password: ")

fdp_client = FDPClient(base_url=fdp_base, username=username, password=password)

In [3]:
class FDPCatalog(HRICatalog):
    is_part_of: [AnyHttpUrl] = Field(
        description="Link to parent object", 
        json_schema_extra={
            "rdf_term": DCTERMS.isPartOf, 
            "rdf_type": "uri"
        })

class FDPDatasetSeries(HRIDatasetSeries):
    is_part_of: [AnyHttpUrl] = Field(
        description="Link to parent object", 
        json_schema_extra={
            "rdf_term": DCTERMS.isPartOf, 
            "rdf_type": "uri"
        })

class FDPProject(HRIProject):
    is_part_of: [AnyHttpUrl] = Field(
        description="Link to parent object", 
        json_schema_extra={
            "rdf_term": DCTERMS.isPartOf, 
            "rdf_type": "uri"
        })


## Catalog

In [4]:
# Create a class instance with the same data
fdp_catalog = FDPCatalog(
    title=[
        LiteralField(value="Inflammatory Bowel Disease catalogue", language="en")
    ],
    description=[
        LiteralField(value="This catalogue describes the core metadata of AUMC Inflammatory Bowel Disease datasets", language="en")
    ],
    contact_point=HRIVCard(
        hasEmail="mailto:data-access-committee@xumc.nl",
        full_name="Data Access Committee of the x UMC",
        hasUID="https://ror.org/05wg1m734"),
    publisher=HRIAgent(
        name=LiteralField(value="Academic Medical Center"),
        identifier="https://ror.org/05wg1m734",
        homepage=URIRef("https://www.xumc.nl"),
        mbox="mailto:data-access-committee@xumc.nl"
    ),
    is_part_of=[URIRef(fdp_base)],
    dataset=[])

fdp_catalog_record = fdp_catalog.to_graph(URIRef(f"https://www.example.com/catalog/1"))
print(fdp_catalog_record.serialize())


@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix v: <http://www.w3.org/2006/vcard/ns#> .

<https://www.example.com/catalog/1> a dcat:Catalog ;
    dcterms:description "This catalogue describes the core metadata of AUMC Inflammatory Bowel Disease datasets"@en ;
    dcterms:isPartOf <http://localhost:8081> ;
    dcterms:publisher [ a foaf:Agent ;
            dcterms:identifier "https://ror.org/05wg1m734" ;
            foaf:homepage <https://www.xumc.nl/> ;
            foaf:mbox <mailto:data-access-committee@xumc.nl> ;
            foaf:name "Academic Medical Center" ] ;
    dcterms:title "Inflammatory Bowel Disease catalogue"@en ;
    dcat:contactPoint [ a v:Kind ;
            v:fn "Data Access Committee of the x UMC" ;
            v:hasEmail <mailto:data-access-committee@xumc.nl> ;
            v:hasUID <https://ror.org/05wg1m734> ] .




In [5]:
catalog_fdp_url = fdp_client.create_and_publish(resource_type="catalog", metadata=fdp_catalog_record)
print(catalog_fdp_url)

http://localhost:8081/catalog/04d608bb-3273-4b7f-b1cd-bd37a6e5bab3


## Dataset

In [6]:
hri_dataset = HRIDataset(
    contact_point=HRIVCard(
        hasEmail="mailto:data-access-committee@xumc.nl",
        full_name="Data Access Committee of the x UMC", 
        hasUID="https://ror.org/05wg1m734")
    ,
    creator=[HRIAgent(
        name="Academic Medical Center", 
        identifier="https://ror.org/05wg1m734",
        homepage="https://www.xumc.nl",
        mbox="mailto:data-access-committee@xumc.nl"    
    )],
    description=[LiteralField(value=
                              "The primary aim of the PRISMA study was to investigate the potential value of risk-tailored versus "
                              "traditional breast cancer screening protocols in the Netherlands. Data collection took place between "
                              "2014-2019, resulting in ∼67,000 mammograms, ∼38,000 surveys, ∼10,000 blood samples and ∼600 saliva "
                              "samples.")],
    issued=parser.isoparse("2024-07-01T11:11:11"),
    identifier="https://www.example.com/dataset/ZLOYOJ",
    modified=parser.isoparse("2024-06-04T13:36:10.246Z"),
    publisher=HRIAgent(
        name="Academic Medical Center", 
        identifier="https://ror.org/05wg1m734",
        homepage="https://www.xumc.nl",
        mbox="mailto:data-access-committee@xumc.nl"    
    ),
    theme=[URIRef("http://publications.europa.eu/resource/authority/data-theme/HEAL")],
    title=[LiteralField(value="Questionnaire data of the Personalised RISk-based MAmmascreening Study (PRISMA)")],
    distribution=[],
    access_rights=URIRef("http://publications.europa.eu/resource/authority/access-right/RESTRICTED"),
    keyword=['example']
)

In [7]:
fdp_dataset_record = hri_dataset.to_graph(subject=URIRef(hri_dataset.identifier))
fdp_dataset_record.add((
    URIRef(hri_dataset.identifier), 
    DCTERMS.isPartOf, 
    URIRef(catalog_fdp_url)
))
dataset_fdp_url = fdp_client.create_and_publish(resource_type="dataset", metadata=fdp_dataset_record)

print(dataset_fdp_url)

http://localhost:8081/dataset/c9ff0287-9a81-4159-9fb7-0479416e290b


## Distribution

In [8]:
hri_distribution = HRIDistribution(
    title=[
        LiteralField(value="CSV-distribution of the questionnaire data of the Personalised RISk-based MAmmascreening Study (PRISMA)")
    ],
    description=[
        LiteralField(value="CSV file containing the questionnaire data of the PRISMA study")
    ],
    access_url=URIRef("https://example.com/dataset/PRISMA/questionnaire.csv"),
    media_type=URIRef("https://www.iana.org/assignments/media-types/text/csv"),
    byte_size=4096,
    license=URIRef("https://definities.geostandaarden.nl/dcat-ap-nl/id/waardelijst/licenties/niet_open"),
    rights="https://www.example.com/contracts/definitely_a_real_DPA.pdf",
    format=URIRef("http://publications.europa.eu/resource/authority/file-type/CSV")
)

In [9]:
access_url_str = str(hri_distribution.access_url)
distribution_uri = URIRef(f"{hri_dataset.identifier}/distribution/{access_url_str.split('/')[-1]}")
fdp_distribution_record = hri_distribution.to_graph(subject=distribution_uri)
fdp_distribution_record.add((distribution_uri, DCTERMS.isPartOf, URIRef(f"{dataset_fdp_url}")))
distribution_fdp_url = fdp_client.create_and_publish(resource_type="distribution", metadata=fdp_distribution_record)

print(distribution_fdp_url)

http://localhost:8081/distribution/f6a62ae4-b68c-4489-810e-ab84c7e2f99b


## Data Service

In [10]:
hri_dataservice = HRIDataService(
    title=[
        LiteralField(value="Example Data Service Title")
    ],
    description=[
        LiteralField(value="Example Data Service Description")
    ],
    endpoint_description=LiteralField(value="Example Data Service Endpoint Description"),
    contact_point=HRIVCard(
        hasEmail="mailto:data-access-committee@xumc.nl",
        full_name="Data Access Committee of the x UMC", 
        hasUID="https://ror.org/05wg1m734"),
    access_rights=URIRef("http://publications.europa.eu/resource/authority/access-right/RESTRICTED"),
    endpoint_url="https://www.example.com/dataservice/1",
    identifier="www.example.com/dataservice/1",
    license=URIRef("https://definities.geostandaarden.nl/dcat-ap-nl/id/waardelijst/licenties/niet_open"),
    publisher=HRIAgent(
        name="Academic Medical Center", 
        identifier="https://ror.org/05wg1m734",
        homepage="https://www.xumc.nl",
        mbox="mailto:data-access-committee@xumc.nl"    
    ),
    theme=[URIRef("http://publications.europa.eu/resource/authority/data-theme/HEAL")]
)

In [11]:
fdp_dataservice_record = hri_dataservice.to_graph(subject=URIRef(hri_dataservice.identifier))
fdp_dataservice_record.add((
    URIRef(hri_dataservice.identifier), 
    DCTERMS.isPartOf, 
    URIRef(f"{catalog_fdp_url}")
))
dataservice_fdp_url = fdp_client.create_and_publish(resource_type="dataservice", metadata=fdp_dataservice_record)

print(dataservice_fdp_url)

http://localhost:8081/dataservice/52501b33-3a0d-4b00-9341-67f2f5d4cd0f


## Dataset Series

In [12]:
fdp_datasetseries = FDPDatasetSeries(
    title=[
        LiteralField(value="Example Dataset Series title")
    ],
    description=[
        LiteralField(value="Example Dataset Series description")
    ],
    is_part_of=[URIRef(fdp_base)],
)
fdp_datasetseries_record = fdp_datasetseries.to_graph(URIRef(f"https://www.example.com/datasetseries/1"))
print(fdp_datasetseries_record.serialize())

@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dcterms: <http://purl.org/dc/terms/> .

<https://www.example.com/datasetseries/1> a dcat:DatasetSeries ;
    dcterms:description "Example Dataset Series description" ;
    dcterms:isPartOf <http://localhost:8081> ;
    dcterms:title "Example Dataset Series title" .




In [13]:
datasetseries_fdp_url = fdp_client.create_and_publish(resource_type="datasetseries", metadata=fdp_datasetseries_record)

print(datasetseries_fdp_url)

http://localhost:8081/datasetseries/bde492f9-5549-43d0-8709-10aaf034a4bc


## Project

In [14]:
fdp_project = FDPProject(
    title=[
        LiteralField(value="Example Project title")
    ],
    description=[
        LiteralField(value="Example Project description")
    ],
    identifier="https://www.example.com/project/1",
    funder=[HRIAgent(
        name="Example Funder", 
        identifier="https://ror.org/05wg1m734",
        homepage="https://www.xumc.nl",
        mbox="mailto:data-access-committee@xumc.nl"    
    )],
    is_part_of=[URIRef(fdp_base)],
    catalog=[URIRef(catalog_fdp_url)]
)
fdp_project_record = fdp_project.to_graph(URIRef(f"https://www.example.com/project/1"))
print(fdp_project_record.serialize())

@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://www.example.com/project/1> a foaf:Project ;
    dcterms:description "Example Project description" ;
    dcterms:identifier "https://www.example.com/project/1"^^xsd:string ;
    dcterms:isPartOf <http://localhost:8081> ;
    dcterms:title "Example Project title" ;
    dcat:resource <http://localhost:8081/catalog/04d608bb-3273-4b7f-b1cd-bd37a6e5bab3> ;
    foaf:fundedBy [ a foaf:Agent ;
            dcterms:identifier "https://ror.org/05wg1m734" ;
            foaf:homepage <https://www.xumc.nl/> ;
            foaf:mbox <mailto:data-access-committee@xumc.nl> ;
            foaf:name "Example Funder" ] .




In [15]:
project_fdp_url = fdp_client.create_and_publish(resource_type="project", metadata=fdp_project_record)

print(project_fdp_url)

http://localhost:8081/project/b32d89fd-c230-4ec0-9d1d-b5ef68445fba


## Study

In [20]:
fdp_study = HRIStudy(
    title=[
        LiteralField(value="Example Study title")
    ],
    description=[
        LiteralField(value="Example Study description")
    ],
    dataset=[
        URIRef(dataset_fdp_url)
    ],
    identifier="https://www.example.com/study/1",
    project=URIRef(project_fdp_url),
)
fdp_study_record = fdp_study.to_graph(URIRef(f"https://www.example.com/study/1"))
print(fdp_study_record.serialize())

@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix disco: <http://rdf-vocabulary.ddialliance.org/discovery#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://www.example.com/study/1> a disco:Study ;
    dcterms:description "Example Study description" ;
    dcterms:identifier "https://www.example.com/study/1"^^xsd:string ;
    dcterms:isPartOf <http://localhost:8081/project/b32d89fd-c230-4ec0-9d1d-b5ef68445fba> ;
    dcterms:title "Example Study title" ;
    prov:generated <http://localhost:8081/dataset/c9ff0287-9a81-4159-9fb7-0479416e290b> .




In [21]:
study_fdp_url = fdp_client.create_and_publish(resource_type="study", metadata=fdp_study_record)

print(study_fdp_url)

http://localhost:8081/study/c2ad0437-c0ed-42ab-8a32-6ad454f7ff71
